# 3D Information Processing
Obtain a 3D shape from images taken by multiple cameras

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from camera import Camera
from stereo import Stereo
from visualize import plot_calibration_points

In [3]:
# 指数表記にしない
np.set_printoptions(suppress=True)

Load calibration points for each images

In [4]:
points_1 = pd.read_csv("points/points_1.csv")
points_2 = pd.read_csv("points/points_2.csv")
points_3 = pd.read_csv("points/points_3.csv")

Plot calibration points

In [5]:
plot_calibration_points("data/1.JPG", "data/1_plotted.JPG", points_1)
plot_calibration_points("data/2.JPG", "data/2_plotted.JPG", points_2)
plot_calibration_points("data/3.JPG", "data/3_plotted.JPG", points_3)

### 1. Obtain perspective projection matrix

Calibrate cameras from calibration points

In [6]:
c1 = Camera(points_1)
c2 = Camera(points_2)
c3 = Camera(points_3)

In [7]:
c1.calibrate()
c2.calibrate()
c3.calibrate()

Calibrate camera...
Perspective Projection Matrix
[[ -40.54122968   40.37259325  -22.09534006 1571.80271692]
 [  14.67419715    5.29694456  -55.32118497 1118.37794573]
 [  -0.0117374    -0.0041147    -0.01115845    1.        ]]
Calibrate camera...
Perspective Projection Matrix
[[ -47.5848755    31.63423375  -18.0047358  1582.83552373]
 [   6.04330571    2.71247349  -55.79891734 1083.35787306]
 [  -0.01209186   -0.00646705   -0.00886434    1.        ]]
Calibrate camera...
Perspective Projection Matrix
[[ -46.84682189   27.79618015  -23.94934871 1673.69448202]
 [  16.76095364   12.07519625  -51.80189787  889.83455189]
 [  -0.00888891   -0.00640756   -0.01209724    1.        ]]


Validate perspective projection matrix by other points

In [8]:
points_valid = pd.read_csv("points/points_1_valid.csv")
pred_u = []
pred_v = []
for row in points_valid.itertuples():
    u_, v_ = c1.perspective_project(row[3], row[4], row[5])
    pred_u += [u_]
    pred_v += [v_]

predict = pd.DataFrame({"u": pred_u, "v": pred_v,
                        "x": points_valid["x"],
                        "y": points_valid["y"],
                        "z": points_valid["z"]})

predict["u"] = predict["u"].astype(int)
predict["v"] = predict["v"].astype(int)

plot_calibration_points("data/1.JPG", "data/1_pred.JPG", points_valid)
plot_calibration_points("data/1_pred.JPG", "data/1_pred.JPG",
                        predict, color=(0, 0, 230))

Re-project calibration points

In [9]:
c1.re_project()

plot_calibration_points("data/1.JPG", "data/1_reproject.JPG",
                        c1.points_for_calibration)
plot_calibration_points("data/1_reproject.JPG", "data/1_reproject.JPG",
                        c1.points_of_reprojection,
                        color=(0, 0, 230))
reprojection_error = c1.re_projection_error()
reprojection_error

6.332554156756919

In [10]:
c2.re_project()

plot_calibration_points("data/2.JPG", "data/2_reproject.JPG",
                        c2.points_for_calibration)
plot_calibration_points("data/2_reproject.JPG", "data/2_reproject.JPG",
                        c2.points_of_reprojection,
                        color=(0, 0, 230))

reprojection_error = c2.re_projection_error()
reprojection_error

5.146138217502861

In [11]:
c3.re_project()

plot_calibration_points("data/3.JPG", "data/3_reproject.JPG",
                        c3.points_for_calibration)
plot_calibration_points("data/3_reproject.JPG", "data/3_reproject.JPG",
                        c3.points_of_reprojection,
                        color=(0, 0, 230))

reprojection_error = c3.re_projection_error()
reprojection_error

5.419570702840029

### 2. Obtain 3D points from 2 images

Obtain 3D points from 2 images

In [12]:
points12 = pd.read_csv("points/points_1_2.csv")
s = Stereo(c1, c2, points12)
s.obtain_objects_points_by_stereo()

(X, Y, Z) =  [25.46853018 16.45891675  0.27088423]
(X, Y, Z) =  [24.74322514 17.17460602  0.21459361]
(X, Y, Z) =  [21.9623529  20.02167334  0.32099857]
(X, Y, Z) =  [19.91808882 22.15987022  0.38459758]
(X, Y, Z) =  [16.4237593  25.69960559  0.35387484]
(X, Y, Z) =  [5.25763004 7.23655657 9.16204878]
(X, Y, Z) =  [10.59512338  5.50404891  9.27484466]
(X, Y, Z) =  [9.37281706 8.25184944 9.13307199]
(X, Y, Z) =  [ 8.15256931 11.08960475  9.13429177]
(X, Y, Z) =  [4.45047282 9.9362386  9.156412  ]
(X, Y, Z) =  [11.59571684  6.51458292  2.8439244 ]
(X, Y, Z) =  [10.64963668 10.55205759  6.59275462]
(X, Y, Z) =  [11.65461478  6.63672553  6.33230488]
(X, Y, Z) =  [ 9.52752728 11.54604342  8.79831639]
(X, Y, Z) =  [ 9.39823315 11.52663161  2.80247845]
(X, Y, Z) =  [13.09258375 17.84186136  6.14589095]
(X, Y, Z) =  [15.30964117 17.3783268   1.98313607]
(X, Y, Z) =  [13.30651068 21.42861199  2.55998543]
(X, Y, Z) =  [18.57716264 12.74336519  6.16123568]
(X, Y, Z) =  [ 8.38610995 27.03306098  9

In [13]:
output_stereo = pd.concat([points12, s.points_of_objects], axis=1)
output_stereo.to_csv("points/output_points_1_2.csv", float_format="%.6f")

s.points_of_objects

,X,Y,Z
0,25.468530,16.458917,0.270884
1,24.743225,17.174606,0.214594
2,21.962353,20.021673,0.320999
3,19.918089,22.159870,0.384598
4,16.423759,25.699606,0.353875
5,5.257630,7.236557,9.162049
6,10.595123,5.504049,9.274845
7,9.372817,8.251849,9.133072
8,8.152569,11.089605,9.134292
9,4.450473,9.936239,9.156412


In [14]:
points_1_ = pd.DataFrame({"u": points12["u1"], "v": points12["v1"],
                          "x": s.points_of_objects["X"],
                          "y": s.points_of_objects["Y"],
                          "z": s.points_of_objects["Z"]})
points_2_ = pd.DataFrame({"u": points12["u2"], "v": points12["v2"],
                          "x": s.points_of_objects["X"],
                          "y": s.points_of_objects["Y"],
                          "z": s.points_of_objects["Z"]})

plot_calibration_points("data/1.JPG", "data/1_stereo.JPG", points_1_)
plot_calibration_points("data/2.JPG", "data/2_stereo.JPG", points_2_)

Re-project from obatined 3D points

In [15]:
re_points_1_ = c1.perspective_project_points(points_1_)
re_points_2_ = c2.perspective_project_points(points_2_)
re_points_1_

,u,v,x,y,z
0,1900,2481,25.468530,16.458917,0.270884
1,1975,2451,24.743225,17.174606,0.214594
2,2259,2329,21.962353,20.021673,0.320999
3,2460,2246,19.918089,22.159870,0.384598
4,2775,2115,16.423759,25.699606,0.353875
5,1796,901,5.257630,7.236557,9.162049
6,1547,1053,10.595123,5.504049,9.274845
7,1754,1053,9.372817,8.251849,9.133072
8,1965,1045,8.152569,11.089605,9.134292
9,1976,906,4.450473,9.936239,9.156412


In [16]:
plot_calibration_points("data/1.JPG", "data/1_stereo_reproject.JPG", points_1_)
plot_calibration_points("data/1_stereo_reproject.JPG", "data/1_stereo_reproject.JPG",
                        re_points_1_, color=(0, 0, 230))

plot_calibration_points("data/2.JPG", "data/2_stereo_reproject.JPG", points_2_)
plot_calibration_points("data/2_stereo_reproject.JPG", "data/2_stereo_reproject.JPG",
                        re_points_2_, color=(0, 0, 230))

### 3. Points on the carpenter's square

In [17]:
# 3次元中の距離を計算
def get_distance_on_object_points(X1, X2):
    x1, y1, z1 = X1
    x2, y2, z2 = X2

    d = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)
    return d

In [18]:
# 差金上の点は0-4まで
points_on_carpenter = s.points_of_objects.iloc[:5, :]
points_on_carpenter

,X,Y,Z
0,25.468530,16.458917,0.270884
1,24.743225,17.174606,0.214594
2,21.962353,20.021673,0.320999
3,19.918089,22.159870,0.384598
4,16.423759,25.699606,0.353875


In [19]:
distance_on_carpenter = []

for i in range(len(points_on_carpenter) - 1):
    x1, y1, z1 = points_on_carpenter.iat[i, 0], points_on_carpenter.iat[i, 1], points_on_carpenter.iat[i, 2]
    x2, y2, z2 = points_on_carpenter.iat[i + 1, 0], points_on_carpenter.iat[i + 1, 1], points_on_carpenter.iat[i + 1, 2]

    d = get_distance_on_object_points((x1, y1, z1), (x2, y2, z2))
    distance_on_carpenter += [d]

distance_on_carpenter

[1.0205131877439537, 3.9812516550414876, 2.958875862963973, 4.974033488485085]

#### Line in Space

##### 1. Assuming z=0, find y=ax+b by the least squares method

In [20]:
def calc_x_y_line_by_least_squares(points: pd.DataFrame):
    x_sq = [x ** 2 for x in points["X"]]
    x_y = [x * y for x, y in zip(points["X"], points["Y"])]

    array = [
        [sum(x_sq), sum(points["X"])],
        [sum(points["X"]), 1 * len(points)]
    ]

    vector = [
        sum(x_y),
        sum(points["Y"])
    ]

    array = np.array(array)
    vector = np.array(vector)

    x_ = np.linalg.solve(array, vector)
    a = x_[0]
    b = x_[1]

    return a, b

In [21]:
a, b = calc_x_y_line_by_least_squares(points_on_carpenter)

In [22]:
distance_to_line = []

for row in points_on_carpenter.itertuples():
    x, y = row[1], row[2]
    d = np.abs(a * x - y + b) / np.sqrt(a ** 2 + b ** 2)
    distance_to_line += [d]

distance_to_line

[0.00028853920314414125,
 0.00034931253276076335,
 0.0003723134313846968,
 0.0006749264699789848,
 0.00024183970897766595]

##### 2. Line connecting point 1 and point 5

In [23]:
distance_to_line = []

vector_a = points_on_carpenter.iloc[0, :3] # 直線上の点
vector_a = np.array(vector_a)
vector_l = points_on_carpenter.iloc[4, :3]
vector_l = np.array(vector_l)
vector_u = vector_l - vector_a # 方向ベクトル

for row in points_on_carpenter.itertuples():
    vector_p = [row[1], row[2], row[3]]
    vector_p = np.array(vector_p)
    vector_ap = vector_p - vector_a
    d = np.linalg.norm(np.cross(vector_ap, vector_u)) / np.linalg.norm(vector_u)
    distance_to_line += [d]

distance_to_line

[0.0, 0.06527810583599233, 0.022553917016332272, 0.06612965283865363, 0.0]

##### 3. Find two planes by least squares method, and find the intersection line

In [24]:
def calc_line_by_2_plane_by_least_square(points: pd.DataFrame):
    z_sq = [z ** 2 for z in points["Z"]]
    x_z = [x * z for x, z in zip(points["X"], points["Z"])]
    y_z = [y * z for y, z in zip(points["Y"], points["Z"])]

    # x = az + bを求める
    array_x = [
        [sum(z_sq), sum(points["Z"])],
        [sum(points["Z"]), 1 * len(points)]
    ]

    vector_x = [
        sum(x_z),
        sum(points["X"])
    ]

    array_x = np.array(array_x)
    vector_x = np.array(vector_x)

    x_ = np.linalg.solve(array_x, vector_x)
    a, b = x_[0], x_[1]

    # y = cz + dを求める
    array_y = [
        [sum(z_sq), sum(points["Z"])],
        [sum(points["Z"]), 1 + len(points)]
    ]

    vector_y = [
        sum(y_z),
        sum(points["Y"])
    ]

    array_y = np.array(array_y)
    vector_y = np.array(vector_y)

    y_ = np.linalg.solve(array_y, vector_y)
    c, d = y_[0], y_[1]

    # 方向ベクトルと通る点のベクトルを返す
    vector_u = np.array([a, b, 1.0])
    vector_a = np.array([b, d, 0.0])

    return vector_u, vector_a

In [25]:
vector_u, vector_a = calc_line_by_2_plane_by_least_square(points_on_carpenter)

In [26]:
distance_to_line = []

for row in points_on_carpenter.itertuples():
    vector_p = np.array([row[1], row[2], row[3]])
    vector_ap = vector_p - vector_a
    d = np.linalg.norm(np.cross(vector_ap, vector_u)) / np.linalg.norm(vector_u)
    distance_to_line += [d]

distance_to_line

[5.925169034284086,
 6.029240839672196,
 6.504487627594214,
 6.8894239061461615,
 7.4598411461297705]

### 4. Points on the cylinder

#### 4.1. Points on the top of the cylinder

In [27]:
def calc_plane_by_least_squares(points: pd.DataFrame):
    array = []
    vector = []

    for row in points.itertuples():
        x, y, z = row[1], row[2], row[3]
        array += [[1, x, y]]
        vector += [z]


    array = np.array(array)
    vector = np.array(vector)

    a_t_a = array.T @ array
    a_t_b = array.T @ vector

    x_ = np.linalg.solve(a_t_a, a_t_b)

    return x_

In [28]:
# 円筒上の点は5-9まで
points_on_plane = s.points_of_objects.iloc[5:10, :]

plane_param = calc_plane_by_least_squares(points_on_plane)
plane_param

array([ 9.30262932,  0.00390124, -0.01904065])

In [29]:
distance_to_plane = []

for row in points_on_plane.itertuples():
    x, y, z = row[1], row[2], row[3]
    d = np.abs(plane_param[0] * x + plane_param[1] * y + z - plane_param[2]) / np.sqrt(plane_param[0] ** 2 + plane_param[1] ** 2 + 1 ** 2)
    distance_to_plane += [d]

distance_to_plane

[6.211811976774725,
 11.530064190655928,
 10.300752512098693,
 9.088808253541956,
 5.409801567227988]

In [30]:
np.average(distance_to_plane)

8.508247700059858

#### 4.2. Points on the side of the cylinder

In [31]:
def calc_circle_by_least_squares(points: pd.DataFrame):
    sum_x_sq = [x ** 2 for x in points["X"]]
    sum_y_sq = [y ** 2 for y in points["Y"]]
    sum_x_y = [x * y for x, y in zip(points["X"], points["Y"])]

    array = [
        [sum(sum_x_sq), sum(sum_x_y), sum(points["X"])],
        [sum(sum_x_y), sum(sum_y_sq), sum(points["Y"])],
        [sum(points["X"]), sum(points["Y"]), 1 * len(points)]
    ]

    sum_v_1 = [x ** 3 - x * (y ** 2) for x, y in zip(points["X"], points["Y"])]
    sum_v_2 = [(x ** 2) * y - y ** 3 for x, y in zip(points["X"], points["Y"])]
    sum_v_3 = [x ** 2 - y ** 2 for x, y in zip(points["X"], points["Y"])]

    vector = [
        -1 * sum(sum_v_1),
        -1 * sum(sum_v_2),
        -1 * sum(sum_v_3)
    ]

    array = np.array(array)
    vector = np.array(vector)

    x_ = np.linalg.solve(array, vector)
    a = x_[0] / -2
    b = x_[1] / -2
    r = np.sqrt(a ** 2 + b ** 2)

    return a, b, r

In [32]:
# 円筒の側面の点は10-14まで
points_on_circle = s.points_of_objects.iloc[10:15, :]

a, b, r = calc_circle_by_least_squares(points_on_circle)
r

14.839396002653642

In [33]:
# 円の中心との距離を求める

distance_to_center = []

for row in points_on_circle.itertuples():
    x, y = row[1], row[2]
    d = np.sqrt((x - a) ** 2 + (y - b) ** 2)

    distance_to_center += [d]

distance_to_center

[14.814298261179466,
 18.907706281574995,
 14.933586124376635,
 20.02308027842922,
 20.022270711952405]

In [34]:
# 半径との差をとる
np.abs(distance_to_center - r)

array([0.02509774, 4.06831028, 0.09419012, 5.18368428, 5.18287471])

### 5. Project obtained 3D points from 1.JPG and 2.JPG to 3.JPG

In [35]:
re_point_3 = c3.perspective_project_points(points_1_)
re_point_3

,u,v,x,y,z
0,1401,2258,25.468530,16.458917,0.270884
1,1478,2248,24.743225,17.174606,0.214594
2,1774,2204,21.962353,20.021673,0.320999
3,1992,2175,19.918089,22.159870,0.384598
4,2350,2126,16.423759,25.699606,0.353875
5,1770,742,5.257630,7.236557,9.162049
6,1461,861,10.595123,5.504049,9.274845
7,1652,893,9.372817,8.251849,9.133072
8,1851,921,8.152569,11.089605,9.134292
9,1936,776,4.450473,9.936239,9.156412


In [36]:
points_3_obj = pd.read_csv("points/points_3_object.csv")
points_3_obj = pd.concat([points_3_obj, s.points_of_objects], axis=1)

plot_calibration_points("data/3.JPG", "data/3_stereo_project.JPG", points_3_obj)
plot_calibration_points("data/3_stereo_project.JPG", "data/3_stereo_project.JPG", re_point_3,
                        color=(0, 0, 230))

Real-Projection Error (?)

In [37]:
def real_projection_error(real_points: pd.DataFrame, projection_points: pd.DataFrame):
    errors = []
    for row_real, row_pro in zip(real_points.itertuples(), projection_points.itertuples()):
        error = np.sqrt((row_real[1] - row_pro[1]) ** 2 + (row_real[2] - row_pro[2]) ** 2)
        errors += [error]

    print(errors)
    return np.average(errors)

In [38]:
rpe = real_projection_error(points_3_obj, re_point_3)
rpe

1.4142135623730951
0.0
4.0
6.708203932499369
6.324555320336759
3.605551275463989
2.8284271247461903
3.0
3.1622776601683795
3.0
9.219544457292887
2.23606797749979
2.23606797749979
3.1622776601683795
2.23606797749979
11.045361017187261
2.23606797749979
1.4142135623730951
1.0
2.23606797749979


3.553248273005417